In [1]:
# 获取token
import requests
import os

login_url = "https://admin.summerfarm.net/authentication/auth/username/login"
login_data = {
    "username": "peng.tang@summerfarm.net",
    "password": os.getenv('XIANMU_ADMIN_PASSWORD'),
}

token = requests.post(login_url, data=login_data).json()

print(token)


headers = {
    'token': token.get("data").get("token"),
    'xm-rqid': 'create_fake_merchant_tp',
    'xm-uid': '2047',
    'Content-Type': 'application/json;charset=UTF-8',
}

print(headers)

{'code': 'SUCCESS', 'data': {'authId': 2082, 'bizUserId': 2047, 'phone': '18618107293', 'realname': '唐鹏', 'token': 'admin__2697ea2a-f6fc-4567-90b9-17e523d78231', 'userBaseId': 2080}, 'msg': '', 'success': True}
{'token': 'admin__2697ea2a-f6fc-4567-90b9-17e523d78231', 'xm-rqid': 'create_fake_merchant_tp', 'xm-uid': '2047', 'Content-Type': 'application/json;charset=UTF-8'}


In [2]:
import requests

url = 'https://admin.summerfarm.net/large-area/larges'
large_area = requests.get(url, headers=headers).json()
large_area

{'code': 'SUCCESS',
 'data': [{'areaList': [{'adminId': 1536,
     'administrativeArea': '浙江/杭州市',
     'areaName': '杭州',
     'areaNo': 1001,
     'changeFlag': False,
     'changeStatus': 0,
     'companyAccountId': 107,
     'companyName': '杭州鲜沐科技有限公司',
     'deliveryFee': 10.0,
     'deliveryRule': '{"categoryList":[]}',
     'expressFee': 0.0,
     'id': 2,
     'largeAreaNo': 1,
     'mapSection': '浙江/杭州市/其他区域',
     'memberRule': '[{"grade":0,"threshold":0,"refundAmount":0,"outTimes":0},{"grade":1,"threshold":400,"refundAmount":20,"outTimes":1},{"grade":2,"threshold":2800,"refundAmount":85,"outTimes":3},{"grade":3,"threshold":6000,"refundAmount":180,"outTimes":5}]',
     'parentNo': 1,
     'payChannel': 2,
     'realname': '翟远方',
     'status': True,
     'supportAddOrder': 0},
    {'adminId': 1536,
     'administrativeArea': '浙江/湖州市',
     'areaName': '德清',
     'areaNo': 1002,
     'changeFlag': False,
     'changeStatus': 0,
     'companyAccountId': 107,
     'companyName': 

In [3]:
update_member_grade_json = {
    "areaName": "测试大区域",
    "storeNo": 1,
    "companyAccountId": 1,
    "companyName": "海南水木铭农业科技有限公司",
    "mapSection": [],
    "areaNo": 29316,
    "status": 1,
    "deliveryFee": 10,
    "expressFee": 11,
    "deliveryRule": '{"categoryList":[]}',
    "memberRule": '[{"grade":0,"threshold":0,"refundAmount":0,"outTimes":0},{"grade":1,"threshold":1000,"refundAmount":10,"outTimes":100},{"grade":2,"threshold":2000,"refundAmount":300,"outTimes":200},{"grade":3,"threshold":4000,"refundAmount":500,"outTimes":500}]',
    "freeDay": "",
    "notifyTitle": "",
    "notifyContent": "",
    "notifyRemarks": "",
    "changeFlag": False,
    "changeStatus": 0,
    "changeStoreNo": "",
    "changStartTime": "",
    "administrativeArea": "河南/洛阳市",
    "adminId": 12,
    "supportAddOrder": 1,
    "largeAreaNo": 1,
    "largeAreaName": "测试大区域",
    "largeName": "杭州大区",
    "payChannel": 0,
    "nextDeliveryDate": "",
    "parentNo": 1,
}

In [4]:
import pandas as pd
new_grade_df=pd.read_csv("./新会员等级-分省份2024.csv")
new_grade_df.head(2)

,省份,交易门槛V0,交易门槛V1,交易门槛V2,交易门槛V3,对应运营大区,large_area_no,极速售后额度V0,极速售后额度V1,极速售后额度V2,极速售后额度V3,超时加单次数
0,广东,0,800,2000,5000,广州大区,14,0,30,50,150,0
1,浙江,0,1000,2500,6000,杭州大区,1,0,31,51,151,0


In [5]:
print(new_grade_df['省份'].unique())

['广东' '浙江' '江苏' '湖南' '上海' '湖北' '福建' '四川' '江西' '山东' '重庆' '安徽' '广西' '云南'
 '贵州']


In [6]:
all_sub_area=[]

for large_area_obj in large_area.get('data'):
    print(large_area_obj)
    if int(large_area_obj.get('status'))==0:
        print(f"服务区关闭的:{large_area_obj}")
        continue
    if '测试' in large_area_obj.get('largeAreaName'):
        print(f"服务大区是测试区域:{large_area_obj}")
    for area in large_area_obj.get("areaList"):
        if int(area.get('status'))==0:
            print(f"服务小区是关闭的:{area}")
            continue
        if '测试' in area.get('areaName',''):
            print(f"服务小区是测试区域:{area}")
            continue
        area['largeName']=large_area_obj['largeAreaName']
        all_sub_area.append(area)
    print(all_sub_area[len(all_sub_area)-1])

{'areaList': [{'adminId': 1536, 'administrativeArea': '浙江/杭州市', 'areaName': '杭州', 'areaNo': 1001, 'changeFlag': False, 'changeStatus': 0, 'companyAccountId': 107, 'companyName': '杭州鲜沐科技有限公司', 'deliveryFee': 10.0, 'deliveryRule': '{"categoryList":[]}', 'expressFee': 0.0, 'id': 2, 'largeAreaNo': 1, 'mapSection': '浙江/杭州市/其他区域', 'memberRule': '[{"grade":0,"threshold":0,"refundAmount":0,"outTimes":0},{"grade":1,"threshold":400,"refundAmount":20,"outTimes":1},{"grade":2,"threshold":2800,"refundAmount":85,"outTimes":3},{"grade":3,"threshold":6000,"refundAmount":180,"outTimes":5}]', 'parentNo': 1, 'payChannel': 2, 'realname': '翟远方', 'status': True, 'supportAddOrder': 0}, {'adminId': 1536, 'administrativeArea': '浙江/湖州市', 'areaName': '德清', 'areaNo': 1002, 'changeFlag': False, 'changeStatus': 0, 'companyAccountId': 107, 'companyName': '杭州鲜沐科技有限公司', 'deliveryFee': 10.0, 'deliveryRule': '{"categoryList":[]}', 'expressFee': 0.0, 'id': 3, 'largeAreaNo': 1, 'mapSection': '浙江/湖州市/德清县', 'memberRule': '[

In [7]:
all_sub_area_df=pd.DataFrame(all_sub_area)

In [8]:
import json
new_grade_df.head(2)

new_grade_map={}

def create_member_rule(row):
    grade_json= [
        {"grade": 0, "threshold": row['交易门槛V0'], "refundAmount": row['极速售后额度V0'], "outTimes": row['超时加单次数']},
        {"grade": 1, "threshold": row['交易门槛V1'], "refundAmount": row['极速售后额度V1'], "outTimes": row['超时加单次数']},
        {"grade": 2, "threshold": row['交易门槛V2'], "refundAmount": row['极速售后额度V2'], "outTimes": row['超时加单次数']},
        {"grade": 3, "threshold": row['交易门槛V3'], "refundAmount": row['极速售后额度V3'], "outTimes": row['超时加单次数']}
    ]

    new_grade_map[f"{row['省份']}_{row['对应运营大区']}"]=json.dumps(grade_json)

    return grade_json

new_grade_df['memberRule'] = new_grade_df.apply(create_member_rule, axis=1)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
new_grade_df[['省份','对应运营大区','memberRule']]

,省份,对应运营大区,memberRule
0,广东,广州大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 800, 'refundAmount': 30, 'outTimes': 0}, {'grade': 2, 'threshold': 2000, 'refundAmount': 50, 'outTimes': 0}, {'grade': 3, 'threshold': 5000, 'refundAmount': 150, 'outTimes': 0}]"
1,浙江,杭州大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 1000, 'refundAmount': 31, 'outTimes': 0}, {'grade': 2, 'threshold': 2500, 'refundAmount': 51, 'outTimes': 0}, {'grade': 3, 'threshold': 6000, 'refundAmount': 151, 'outTimes': 0}]"
2,江苏,苏南大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 900, 'refundAmount': 32, 'outTimes': 0}, {'grade': 2, 'threshold': 2400, 'refundAmount': 52, 'outTimes': 0}, {'grade': 3, 'threshold': 5000, 'refundAmount': 152, 'outTimes': 0}]"
3,江苏,苏州大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 900, 'refundAmount': 32, 'outTimes': 0}, {'grade': 2, 'threshold': 2400, 'refundAmount': 52, 'outTimes': 0}, {'grade': 3, 'threshold': 5000, 'refundAmount': 152, 'outTimes': 0}]"
4,湖南,长沙大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 900, 'refundAmount': 33, 'outTimes': 0}, {'grade': 2, 'threshold': 2000, 'refundAmount': 53, 'outTimes': 0}, {'grade': 3, 'threshold': 4000, 'refundAmount': 153, 'outTimes': 0}]"
5,上海,上海大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 1000, 'refundAmount': 34, 'outTimes': 0}, {'grade': 2, 'threshold': 3000, 'refundAmount': 54, 'outTimes': 0}, {'grade': 3, 'threshold': 6000, 'refundAmount': 154, 'outTimes': 0}]"
6,湖北,武汉大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 700, 'refundAmount': 35, 'outTimes': 0}, {'grade': 2, 'threshold': 1800, 'refundAmount': 55, 'outTimes': 0}, {'grade': 3, 'threshold': 4000, 'refundAmount': 155, 'outTimes': 0}]"
7,福建,福州大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 600, 'refundAmount': 36, 'outTimes': 0}, {'grade': 2, 'threshold': 1600, 'refundAmount': 56, 'outTimes': 0}, {'grade': 3, 'threshold': 3000, 'refundAmount': 156, 'outTimes': 0}]"
8,四川,成都大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 700, 'refundAmount': 37, 'outTimes': 0}, {'grade': 2, 'threshold': 2000, 'refundAmount': 57, 'outTimes': 0}, {'grade': 3, 'threshold': 4000, 'refundAmount': 157, 'outTimes': 0}]"
9,江西,长沙大区,"[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 900, 'refundAmount': 38, 'outTimes': 0}, {'grade': 2, 'threshold': 2300, 'refundAmount': 58, 'outTimes': 0}, {'grade': 3, 'threshold': 5000, 'refundAmount': 158, 'outTimes': 0}]"


In [9]:
new_grade_map

{'广东_广州大区': '[{"grade": 0, "threshold": 0, "refundAmount": 0, "outTimes": 0}, {"grade": 1, "threshold": 800, "refundAmount": 30, "outTimes": 0}, {"grade": 2, "threshold": 2000, "refundAmount": 50, "outTimes": 0}, {"grade": 3, "threshold": 5000, "refundAmount": 150, "outTimes": 0}]',
 '浙江_杭州大区': '[{"grade": 0, "threshold": 0, "refundAmount": 0, "outTimes": 0}, {"grade": 1, "threshold": 1000, "refundAmount": 31, "outTimes": 0}, {"grade": 2, "threshold": 2500, "refundAmount": 51, "outTimes": 0}, {"grade": 3, "threshold": 6000, "refundAmount": 151, "outTimes": 0}]',
 '江苏_苏南大区': '[{"grade": 0, "threshold": 0, "refundAmount": 0, "outTimes": 0}, {"grade": 1, "threshold": 900, "refundAmount": 32, "outTimes": 0}, {"grade": 2, "threshold": 2400, "refundAmount": 52, "outTimes": 0}, {"grade": 3, "threshold": 5000, "refundAmount": 152, "outTimes": 0}]',
 '江苏_苏州大区': '[{"grade": 0, "threshold": 0, "refundAmount": 0, "outTimes": 0}, {"grade": 1, "threshold": 900, "refundAmount": 32, "outTimes": 0}, {"

In [10]:
all_sub_area[0]
not_found_areas = []
all_sql=""
for area in all_sub_area:

    area_of_this = f"{area['areaName']}:{area['areaNo']}"
    province = area["administrativeArea"].split("/")[0]
    if len(province) > 3:
        province = province[:2]
    large_name = area["largeName"]
    # print(province, large_name)
    new_grade_json = new_grade_map.get(f"{province}_{large_name}", "呵呵获取不到新规则")
    if "呵呵" in new_grade_json:
        # print(f"本运营服务区找不到新会员规则: {area}")
        not_found_areas.append(area)
        continue

    new_grade_json = json.loads(new_grade_json)
    old_rule = area["memberRule"]
    old_rule_json = None
    if old_rule is not None and len(old_rule) > 10:
        old_rule_json = json.loads(old_rule)
        # print(old_rule_json)
        for rule in new_grade_json:
            # print(f"rule:{rule}")
            for old in old_rule_json:
                # print(f"old:{old}")
                if rule["grade"] == old["grade"]:
                    rule["outTimes"] = old["outTimes"]
                    break
    print(
        f"======>\n运营服务区：{area_of_this},{province},{large_name}, \n旧规则：{old_rule_json}, \n新规则：{new_grade_json}\n\n"
    )
    all_sql=f"""{all_sql}update `area` set `member_rule`='{json.dumps(new_grade_json)}',update_time=now() 
where area_no = {area['areaNo']} and area_name='{area['areaName']}' and large_area_no={area['largeAreaNo']};\n\n"""

not_found_areas_df = pd.DataFrame(not_found_areas)
not_found_areas_df[["administrativeArea", "areaName", "areaNo", "largeName"]]

print("以下是找不到新规则的运营服务区！")
for index, row in not_found_areas_df[["administrativeArea", "areaName", "areaNo", "largeName"]].iterrows():
    print(f"找不到新规则:{row.to_dict()}")

======>
运营服务区：杭州:1001,浙江,杭州大区, 
旧规则：[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 400, 'refundAmount': 20, 'outTimes': 1}, {'grade': 2, 'threshold': 2800, 'refundAmount': 85, 'outTimes': 3}, {'grade': 3, 'threshold': 6000, 'refundAmount': 180, 'outTimes': 5}], 
新规则：[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 1000, 'refundAmount': 31, 'outTimes': 1}, {'grade': 2, 'threshold': 2500, 'refundAmount': 51, 'outTimes': 3}, {'grade': 3, 'threshold': 6000, 'refundAmount': 151, 'outTimes': 5}]


======>
运营服务区：德清:1002,浙江,杭州大区, 
旧规则：[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 400, 'refundAmount': 20, 'outTimes': 1}, {'grade': 2, 'threshold': 2800, 'refundAmount': 85, 'outTimes': 3}, {'grade': 3, 'threshold': 6000, 'refundAmount': 180, 'outTimes': 5}], 
新规则：[{'grade': 0, 'threshold': 0, 'refundAmount': 0, 'outTimes': 0}, {'grade': 1, 'threshold': 1000, 'refundAm

In [12]:
not_found_areas_df[["administrativeArea", "areaName", "areaNo", "largeName"]]

,administrativeArea,areaName,areaNo,largeName
0,江苏/南通市,南通,20882,杭州大区
1,云南/昆明市,昆明快递业务,44209,昆明快递大区
2,河南/许昌市,切仓运营区一,44229,测试大区
3,山西/太原市,太原,13351,山西大区
4,广东/汕头市,广州转转干净未用,44210,废区集散地
5,四川/成都市,茶百道渝六十运费,44193,重庆大区
6,浙江/杭州市,美团虚拟区域一,44217,美团和喜茶大区
7,广东/广州市,喜茶虚拟区域一,44230,美团和喜茶大区
8,黑龙江/哈尔滨市,自营奶哈尔滨,44221,自营奶虚拟大区
9,浙江/杭州市,柠季嘉兴仓快递区域,44226,柠季快递大区


In [11]:
print("SQL：{}")
print(all_sql)

SQL：{}
update `area` set `member_rule`='[{"grade": 0, "threshold": 0, "refundAmount": 0, "outTimes": 0}, {"grade": 1, "threshold": 1000, "refundAmount": 31, "outTimes": 1}, {"grade": 2, "threshold": 2500, "refundAmount": 51, "outTimes": 3}, {"grade": 3, "threshold": 6000, "refundAmount": 151, "outTimes": 5}]',update_time=now() 
where area_no = 1001 and area_name='杭州' and large_area_no=1;

update `area` set `member_rule`='[{"grade": 0, "threshold": 0, "refundAmount": 0, "outTimes": 0}, {"grade": 1, "threshold": 1000, "refundAmount": 31, "outTimes": 1}, {"grade": 2, "threshold": 2500, "refundAmount": 51, "outTimes": 3}, {"grade": 3, "threshold": 6000, "refundAmount": 151, "outTimes": 5}]',update_time=now() 
where area_no = 1002 and area_name='德清' and large_area_no=1;

update `area` set `member_rule`='[{"grade": 0, "threshold": 0, "refundAmount": 0, "outTimes": 0}, {"grade": 1, "threshold": 1000, "refundAmount": 31, "outTimes": 1}, {"grade": 2, "threshold": 2500, "refundAmount": 51, "outT